In [34]:
from sec_api import QueryApi, ExtractorApi
import json
import requests
import pandas as pd
from IPython.display import display, HTML

queryAPI = QueryApi(api_key='041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550')
extractorApi = ExtractorApi(api_key='041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550')

In [35]:
#formType:(“10-K”, “10-KT”, “10KSB”, “10KT405”, “10KSB40”, “10-K405”)
class SEC_QUERY:

    def __init__(self, form_type, ticker, size):
        self.form_type = form_type
        self.ticker = ticker
        self.size = size
        self.query = {
            "query": { 
                "query_string" : {
                    "query": f"formType:\"{self.form_type}\" AND ticker:{self.ticker}",
                }
            },
            "from": "0",
            "size": size,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
    
    def get_response(self, index:int):
        response = queryAPI.get_filings(self.query)
        return json.dumps(response["filings"][index], indent=2)
    

    def extract(self, index:int):
        query_str = self.get_response(index)
        q_dict = json.loads(query_str)
        filing_url = q_dict['linkToHtml']

        xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
        api_key = '041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550'

        final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
        response = requests.get(final_url)

        final_json = json.loads(response.text)
        return final_json, filing_url

In [42]:
query1 = SEC_QUERY("10-K", "TSLA", "10")
#print(query1.get_response(0))
init_query, f_url = query1.extract(0)

# 10-K supported items [1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15]
section_text = extractorApi.get_section(f_url, "8", "text")
section_html = extractorApi.get_section(f_url, "8", 'html')

In [68]:
csv = init_query['StatementsOfIncome']
columns = list(csv.keys())
#values = [[entry.get('value', None) for entry in entry_list] for entry_list in csv.values()]
values = []

for entry_list in csv.values():
    entry_values = [entry.get('value', None) for entry in entry_list]
    values.append(entry_values)

values_transposed = list(map(list, zip(*values)))

df_income = pd.DataFrame(values_transposed, columns=columns)
df_income

,RevenueFromContractWithCustomerExcludingAssessedTax,CostOfRevenue,GrossProfit,ResearchAndDevelopmentExpense,SellingGeneralAndAdministrativeExpense,RestructuringAndOtherExpenses,OperatingExpenses,OperatingIncomeLoss,InvestmentIncomeInterest,InterestExpense,OtherNonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,ProfitLoss,NetIncomeLossAttributableToNoncontrollingInterest,NetIncomeLoss,EarningsPerShareBasic,EarningsPerShareDiluted,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfDilutedSharesOutstanding
0,78509000000,65121000000,17660000000,3969000000,4800000000,0,8769000000,8891000000,1066000000,156000000,172000000,9973000000,-5001000000,14974000000,-23000000,14997000000,4.73,4.30,3174000000,3485000000
1,67210000000,49599000000,20853000000,3075000000,3946000000,176000000,7197000000,13656000000,297000000,191000000,-43000000,13719000000,1132000000,12587000000,31000000,12556000000,4.02,3.62,3130000000,3475000000
2,44125000000,32415000000,13606000000,2593000000,4517000000,-27000000,7083000000,6523000000,56000000,371000000,135000000,6343000000,699000000,5644000000,125000000,5519000000,1.87,1.63,2959000000,3386000000


In [45]:
def pprint(text, line_length=100):
    words = text.split(' ')
    lines = []
    current_line = ' '
    for word in words:
        if len(current_line + ' '+ word) <= line_length:
            current_line += ' '+word
        else:
            lines.append(current_line.strip())
            current_line = word
    if current_line:
        lines.append(current_line.strip())
    print('\n'.join(lines))

In [47]:
pprint(section_text[0:3000])

ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA 

Index to Consolidated Financial Statements
##TABLE_START Page 

Report of Independent Registered Public Accounting Firm (PCAOB ID: 238 )
Consolidated Balance Sheets 

Consolidated Statements of Operations 

Consolidated Statements of
Comprehensive Income 

Consolidated Statements of Redeemable Noncontrolling Interests and Equity
Consolidated Statements of Cash Flows 

Notes to Consolidated Financial Statements
##TABLE_END

Report of Independent Registered Public Accounting Firm 

To the Board of Directors
and Stockholders of Tesla, Inc. 

Opinions on the Financial Statements and Internal Control over
Financial Reporting 

We have audited the accompanying consolidated balance sheets of Tesla, Inc.
and its subsidiaries (the &#8220;Company&#8221;) as of December 31, 2023 and 2022, and the related
consolidated statements of operations, of comprehensive income, of redeemable noncontrolling
interests and equity and of cash flows for each 

In [ ]:
display(HTML(section_html[0:150000]))

In [31]:
tables = pd.read_html(section_html)

In [33]:
df = tables[0]
df

,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Page,Page,Page
2,Report of Independent Registered Public Accoun...,Report of Independent Registered Public Accoun...,Report of Independent Registered Public Accoun...,47,47,47
3,Consolidated Balance Sheets,Consolidated Balance Sheets,Consolidated Balance Sheets,49,49,49
4,Consolidated Statements of Operations,Consolidated Statements of Operations,Consolidated Statements of Operations,50,50,50
5,Consolidated Statements of Comprehensive Income,Consolidated Statements of Comprehensive Income,Consolidated Statements of Comprehensive Income,51,51,51
6,Consolidated Statements of Redeemable Noncontr...,Consolidated Statements of Redeemable Noncontr...,Consolidated Statements of Redeemable Noncontr...,52,52,52
7,Consolidated Statements of Cash Flows,Consolidated Statements of Cash Flows,Consolidated Statements of Cash Flows,53,53,53
8,Notes to Consolidated Financial Statements,Notes to Consolidated Financial Statements,Notes to Consolidated Financial Statements,54,54,54
